In [894]:
import pandas as pd
import numpy as np
from pulp import *
import json
import requests
import time

In [895]:
url = 'https://fantasy.premierleague.com/drf/bootstrap-static'
r = ''
while r == '':
    try:
        r = requests.get(url)
    except:
        time.sleep(5)
        continue
r = requests.get(url)
data = json.dumps((r.json()["elements"]))
df = pd.read_json(data, orient = 'records')

In [896]:
df = df[(df['chance_of_playing_next_round'] == 100.0)|np.isnan(df['chance_of_playing_next_round'])].reset_index(drop=True)

In [897]:
df = df[['id','element_type', 'now_cost', 'first_name', 'second_name','total_points', 'form']]

In [898]:
df.element_type.replace({1: 'gkp', 2: 'def', 3: 'mid', 4:'fwd'}, inplace = True)



In [899]:
df = pd.get_dummies(df, columns = ['element_type'])


In [900]:
df = df[df.id!=114] #rule out Diego Costa

In [901]:
players = df.index.tolist()

In [902]:
prob = LpProblem("fantasy football create squad",LpMaximize)

In [903]:
players_vars = LpVariable.dicts("plyr", players,0,1,LpInteger)

In [904]:
prob += lpSum([df.total_points[i]*players_vars[i] for i in players]), "total points"

In [905]:
prob += lpSum([df.now_cost[i]*players_vars[i] for i in players]) <= 410,"cost constrain"
prob += lpSum([df.element_type_gkp[i]*players_vars[i] for i in players]) == 1, "1 goalkeepers"
prob += lpSum([df.element_type_def[i]*players_vars[i] for i in players]) == 1, "1 defenders"
prob += lpSum([df.element_type_mid[i]*players_vars[i] for i in players]) == 1, "1 midfielders"
prob += lpSum([df.element_type_fwd[i]*players_vars[i] for i in players]) == 2, "1 forwarders"              

In [906]:
prob.writeLP("FantasyFootball.lp")
prob.solve()


1

In [907]:
print("Status: ", LpStatus[prob.status])
print value(prob.objective)

('Status: ', 'Optimal')
953.0


In [908]:
values = []
variables = {x.name: x.varValue for x in prob.variables()}
for i in players_vars:
    key = players_vars[i]
    values.append(variables[str(key)])
df['chosen'] = pd.Series(values, index = df.index)

In [909]:
df_chosen = df[df.chosen > 0].sort_values('total_points', ascending = False)
df_chosen

,id,now_cost,first_name,second_name,total_points,form,element_type_def,element_type_fwd,element_type_gkp,element_type_mid,chosen
313,388,95,Bamidele,Alli,225,0,0,0,0,1,1.0
228,285,115,Romelu,Lukaku,221,0,0,1,0,0,1.0
189,235,85,Roberto,Firmino,180,0,0,1,0,0,1.0
78,98,65,Gary,Cahill,178,0,1,0,0,0,1.0
56,74,50,Tom,Heaton,149,0,0,0,1,0,1.0


In [910]:
df[df.chosen > 0].now_cost.sum()

410L

In [911]:
df = df[~df.index.isin(df_chosen.index)]

In [912]:
players = df.index.tolist()

prob2 = LpProblem("fantasy football create squad2",LpMaximize)
players_vars = LpVariable.dicts("plyr", players,0,1,LpInteger)

prob2 += lpSum([df.total_points[i]*players_vars[i] for i in players]), "total points"

prob2 += lpSum([df.now_cost[i]*players_vars[i] for i in players]) <= 395,"cost constrain"

prob2 += lpSum([df.element_type_gkp[i]*players_vars[i] for i in players]) == 0, "0 goalkeepers"
prob2 += lpSum([df.element_type_def[i]*players_vars[i] for i in players]) == 3, "3 defenders"
prob2 += lpSum([df.element_type_mid[i]*players_vars[i] for i in players]) == 3, "3 midfielders"
prob2 += lpSum([df.element_type_fwd[i]*players_vars[i] for i in players]) == 0, "1 forwarders"   

prob2.writeLP("FantasyFootball2.lp")
prob2.solve()

print("Status: ", LpStatus[prob2.status])
print value(prob2.objective)

values = []
variables = {x.name: x.varValue for x in prob2.variables()}
for i in players_vars:
    key = players_vars[i]
    values.append(variables[str(key)])
df['chosen2'] = pd.Series(values, index = df.index)

df_chosen2 = df[df.chosen2 > 0].sort_values('total_points', ascending = False)
print df[df['chosen2'] > 0].now_cost.sum()
df_chosen2

('Status: ', 'Optimal')
963.0
395


,id,now_cost,first_name,second_name,total_points,form,element_type_def,element_type_fwd,element_type_gkp,element_type_mid,chosen,chosen2
312,387,95,Christian,Eriksen,218,0,0,0,0,1,0.0,1.0
80,100,70,Marcos,Alonso,177,0,1,0,0,0,0.0,1.0
77,97,65,César,Azpilicueta,170,0,1,0,0,0,0.0,1.0
22,34,50,Charlie,Daniels,134,0,1,0,0,0,0.0,1.0
356,448,60,Robert,Snodgrass,133,0,0,0,0,1,0.0,1.0
330,409,55,Etienne,Capoue,131,0,0,0,0,1,0.0,1.0


In [913]:
df = df[~df.index.isin(df_chosen2.index)]

In [914]:
players = df.index.tolist()

prob3 = LpProblem("fantasy football create squad2",LpMaximize)
players_vars = LpVariable.dicts("plyr", players,0,1,LpInteger)

prob3 += lpSum([df.total_points[i]*players_vars[i] for i in players]), "total points"

prob3 += lpSum([df.now_cost[i]*players_vars[i] for i in players]) <= 195,"cost constrain"

prob3 += lpSum([df.element_type_gkp[i]*players_vars[i] for i in players]) == 1, "1 goalkeepers"
prob3 += lpSum([df.element_type_def[i]*players_vars[i] for i in players]) == 1, "1 defenders"
prob3 += lpSum([df.element_type_mid[i]*players_vars[i] for i in players]) == 1, "1 midfielders"
prob3 += lpSum([df.element_type_fwd[i]*players_vars[i] for i in players]) == 1, "1 forwarders"   

prob3.writeLP("FantasyFootball3.lp")
prob3.solve()

print("Status: ", LpStatus[prob3.status])
print value(prob3.objective)

values = []
variables = {x.name: x.varValue for x in prob3.variables()}
for i in players_vars:
    key = players_vars[i]
    values.append(variables[str(key)])
df['chosen3'] = pd.Series(values, index = df.index)

df_chosen3 = df[df.chosen3 > 0].sort_values('total_points', ascending = False)
print df[df['chosen3'] > 0].now_cost.sum()
df_chosen3

('Status: ', 'Optimal')
428.0
195


,id,now_cost,first_name,second_name,total_points,form,element_type_def,element_type_fwd,element_type_gkp,element_type_mid,chosen,chosen2,chosen3
71,91,60,Sam,Vokes,121,0,0,1,0,0,0.0,0.0,1.0
18,30,45,Artur,Boruc,120,0,0,0,1,0,0.0,0.0,1.0
58,77,45,Ben,Mee,94,0,1,0,0,0,0.0,0.0,1.0
104,127,45,James,McArthur,93,0,0,0,0,1,0.0,0.0,1.0


In [915]:
df_final = pd.concat([df_chosen,df_chosen2,df_chosen3])[['now_cost', 'first_name', 'second_name','total_points','element_type_def','element_type_fwd','element_type_gkp','element_type_mid']]

In [916]:
x = df_final[['element_type_def','element_type_fwd','element_type_gkp','element_type_mid']]
df_final['position'] = x.idxmax(1)
df_final.drop(axis = 1, labels = ['element_type_def','element_type_fwd','element_type_gkp','element_type_mid']).sort_values(['position','total_points'], ascending = False)

,now_cost,first_name,second_name,total_points,position
313,95,Bamidele,Alli,225,element_type_mid
312,95,Christian,Eriksen,218,element_type_mid
356,60,Robert,Snodgrass,133,element_type_mid
330,55,Etienne,Capoue,131,element_type_mid
104,45,James,McArthur,93,element_type_mid
56,50,Tom,Heaton,149,element_type_gkp
18,45,Artur,Boruc,120,element_type_gkp
228,115,Romelu,Lukaku,221,element_type_fwd
189,85,Roberto,Firmino,180,element_type_fwd
71,60,Sam,Vokes,121,element_type_fwd
